In [77]:
import numpy as np
import pandas as pd
import plotly.express as px

In [78]:
data = pd.read_csv("Car_Details_original.csv")

In [79]:
data

,Make,Type,Origin,DriveTrain,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length,MSRP
0,Acura,SUV,Asia,All,3.5,6.0,265,17,23,4451,106,189,36945
1,Acura,Sedan,Asia,Front,2.0,4.0,200,24,31,2778,101,172,23820
2,Acura,Sedan,Asia,Front,2.4,4.0,200,22,29,3230,105,183,26990
3,Acura,Sedan,Asia,Front,3.2,6.0,270,20,28,3575,108,186,33195
4,Acura,Sedan,Asia,Front,3.5,6.0,225,18,24,3880,115,197,43755
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,Volvo,Sedan,Europe,Front,2.4,5.0,197,21,28,3450,105,186,40565
424,Volvo,Sedan,Europe,Front,2.3,5.0,242,20,26,3450,105,186,42565
425,Volvo,Sedan,Europe,Front,2.9,6.0,268,19,26,3653,110,190,45210
426,Volvo,Wagon,Europe,Front,1.9,4.0,170,22,29,2822,101,180,26135


In [80]:
data.isnull().sum()

Make           0
Type           0
Origin         0
DriveTrain     0
EngineSize     0
Cylinders      2
Horsepower     0
MPG_City       0
MPG_Highway    0
Weight         0
Wheelbase      0
Length         0
MSRP           0
dtype: int64

In [81]:
data.Cylinders.mode()

0    6.0
Name: Cylinders, dtype: float64

In [82]:
data.Cylinders.fillna(int(data.Cylinders.mode()), inplace=True)

C:\Users\saira\AppData\Local\Temp\ipykernel_14840\1573878564.py:1: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



In [83]:
data.Cylinders.isnull().sum()

0

In [84]:
data.drop(columns=['Make'], inplace=True)

In [85]:
data

,Type,Origin,DriveTrain,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length,MSRP
0,SUV,Asia,All,3.5,6.0,265,17,23,4451,106,189,36945
1,Sedan,Asia,Front,2.0,4.0,200,24,31,2778,101,172,23820
2,Sedan,Asia,Front,2.4,4.0,200,22,29,3230,105,183,26990
3,Sedan,Asia,Front,3.2,6.0,270,20,28,3575,108,186,33195
4,Sedan,Asia,Front,3.5,6.0,225,18,24,3880,115,197,43755
...,...,...,...,...,...,...,...,...,...,...,...,...
423,Sedan,Europe,Front,2.4,5.0,197,21,28,3450,105,186,40565
424,Sedan,Europe,Front,2.3,5.0,242,20,26,3450,105,186,42565
425,Sedan,Europe,Front,2.9,6.0,268,19,26,3653,110,190,45210
426,Wagon,Europe,Front,1.9,4.0,170,22,29,2822,101,180,26135


In [86]:
iqr = data.MSRP.quantile(0.75) - data.MSRP.quantile(0.25)
bottom = data.MSRP.quantile(0.25) - 1.5 * iqr
top = data.MSRP.quantile(0.75) + 1.5 * iqr # different from the plotly chart

In [87]:
print("iqr - ", iqr)
print("bot - ", bottom)
print("top - ", top)

iqr -  18870.75
bot -  -7971.875
top -  67511.125


In [88]:
px.scatter(data.MSRP)

In [89]:
px.box(data.MSRP)

In [90]:
data = data[data.MSRP < top]

In [91]:
px.box(data.MSRP)
# the top fence value is different in plotly and hence we are seeing additional values above the top fence

In [92]:
columns = data.columns
numerical_columns = data._get_numeric_data().columns
categorical_columns = list(set(columns) - set(numerical_columns))
categorical_columns

['DriveTrain', 'Type', 'Origin']

In [93]:
for column in categorical_columns:
    one_hot = pd.get_dummies(data[column]).replace([True, False], [1, 0])
    data = data.drop(column, axis=1)
    data = data.join(one_hot)

In [94]:
data

,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length,MSRP,All,...,Rear,Hybrid,SUV,Sedan,Sports,Truck,Wagon,Asia,Europe,USA
0,3.5,6.0,265,17,23,4451,106,189,36945,1,...,0,0,1,0,0,0,0,1,0,0
1,2.0,4.0,200,24,31,2778,101,172,23820,0,...,0,0,0,1,0,0,0,1,0,0
2,2.4,4.0,200,22,29,3230,105,183,26990,0,...,0,0,0,1,0,0,0,1,0,0
3,3.2,6.0,270,20,28,3575,108,186,33195,0,...,0,0,0,1,0,0,0,1,0,0
4,3.5,6.0,225,18,24,3880,115,197,43755,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,2.4,5.0,197,21,28,3450,105,186,40565,0,...,0,0,0,1,0,0,0,0,1,0
424,2.3,5.0,242,20,26,3450,105,186,42565,0,...,0,0,0,1,0,0,0,0,1,0
425,2.9,6.0,268,19,26,3653,110,190,45210,0,...,0,0,0,1,0,0,0,0,1,0
426,1.9,4.0,170,22,29,2822,101,180,26135,0,...,0,0,0,0,0,0,1,0,1,0


In [95]:
x = list(set(data.columns) - set(["MSRP"]))
Y = ["MSRP"]

In [96]:
x

['Rear',
 'USA',
 'Wheelbase',
 'MPG_City',
 'Sports',
 'Wagon',
 'Length',
 'Hybrid',
 'Asia',
 'Sedan',
 'All',
 'Truck',
 'Front',
 'EngineSize',
 'Weight',
 'Cylinders',
 'Europe',
 'SUV',
 'Horsepower',
 'MPG_Highway']

In [97]:
Y

['MSRP']

In [98]:
px.imshow(data).show()

In [99]:
from sklearn.linear_model import LinearRegression

In [100]:
model = LinearRegression()

In [101]:
X = data[x].to_numpy()
y = data[Y].to_numpy()

In [102]:
from sklearn.model_selection import train_test_split
x_train, x_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [103]:
model.fit(x_train, Y_train)

LinearRegression()

In [104]:
model.score(x_test, Y_test)

0.8102180982357172